In [15]:
# Magic to automatically update imports if functions in utils are changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from sklearn.model_selection import train_test_split

stores_train = pd.read_csv("data/stores_train.csv")
stores_test = pd.read_csv("data/stores_test.csv")
bus_routes = pd.read_csv("data/busstops_norway.csv")
stores_train, stores_val = train_test_split(stores_train, test_size=0.2, random_state=0)

In [7]:
bus_routes.head()

,busstop_id,stopplace_type,importance_level,side_placement,geometry
0,853cb081-cc32-4880-aa3e-26e96870d874,Plattform og lomme,Mangler viktighetsnivå,LEFT_AND_RIGHT,POINT(6.54064723379766 61.0618887850553)
1,156b052b-2771-497a-b4f4-97fed59e1aca,"Lomme og skilt, ikke plattform",Mangler viktighetsnivå,LEFT_AND_RIGHT,POINT(5.89980086113255 60.1421872817075)
2,7312a280-e14f-4b09-a421-02e8fe1bc63e,NaN,Mangler viktighetsnivå,MIDDLE_LEFT,POINT(10.7781327278563 59.9299988828761)
3,d9cda2c7-355a-49c1-b56c-a33180d2a82e,NaN,Mangler viktighetsnivå,MIDDLE_LEFT,POINT(10.7781496457324 59.9301044643692)
4,f803bcd3-182d-450b-bbb4-113c6ca885c2,NaN,Mangler viktighetsnivå,MIDDLE_RIGHT,POINT(10.7781676444854 59.9302099995898)


In [53]:
stores_train.head()
print(stores_train.size)

133731


In [14]:
print(bus_routes["geometry"].to_numpy()[0][6:-1].split(" "))
    

['6.54064723379766', '61.0618887850553']


In [20]:
def distance_between_bus_stop_and_store(bus_point, lat, lon):
    bus_point = np.array(bus_point[6:-1].split(" ")).astype(float)
    return math.hypot(bus_point[0] - lat, bus_point[1] - lon)

In [21]:
print(distance_between_bus_stop_and_store("POINT(10.7781676444854 59.9302099995898)", 10.7, 59.9))

0.0838022954316392


In [51]:
def closest_bus_stop(stores: pd.DataFrame, busses: pd.DataFrame):
    stores["closest_bus_stop"] = stores.apply(lambda store_row: min([distance_between_bus_stop_and_store(row, store_row.lat, store_row.lon) for (columnName, row) in busses["geometry"].iteritems()]), axis=1)
    

In [52]:
closest_bus_stop(stores_train, bus_routes)

stores_train.head()

Index(['store_id', 'year', 'store_name', 'plaace_hierarchy_id',
       'sales_channel_name', 'grunnkrets_id', 'address', 'lat', 'lon',
       'chain_name', 'mall_name', 'revenue'],
      dtype='object')


,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue,closest_bus_stop
2506,990857423-990974489-88185,2016,BAKER KLAUSEN UNION,1.2.2.0,Bakery shop,6020601,GRØNLAND 54,59.743135,10.194627,BAKER KLAUSEN DRAMMEN,NaN,4.447,66.280115
12305,948879220-972145203-18800,2016,PRIKKEN RENS AS,3.4.2.0,Laundromats and dry cleaners,2350203,ENERGIVEGEN 5,60.149288,11.187290,NaN,NaN,2.040,65.579151
4142,951136751-971668970-6795,2016,EXPERT HOKKSUND,2.3.1.0,Electronics stores,6240307,SUNDMOEN NÆRINGSOMRÅDE,59.768986,9.899066,EXPERT,NaN,36.489,66.556084
8561,980237508-980246167-71346,2016,INVIT AS,2.9.1.0,Gifts and interior design shops,15040206,APOTEKERGATA 9,62.471607,6.151301,NaN,NaN,5.739,71.119809
818,974421305-974489902-136560,2016,MUCHO MAS,1.1.6.2,Restaurants,3012312,THORVALD MEYERS GATE 36,59.924205,10.759401,NaN,NaN,15.573,65.857399
